<a href="https://colab.research.google.com/github/dbabrams/ShallowDolomite_Group/blob/PumpingData%2Fcharlotte/PumpingDataFilled_4.15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import urllib, json

In [9]:
#This line will have the full data set printed out. 
#Toggle off and clear output if you no longer want to see full dataset
pd.set_option("display.max_rows", None, "display.max_columns", None)

#raw_data provides all of the data from the CSV file
#df1 drops deep wells and those with unknown locations or depths
#read in the csv
raw_csv = 'https://raw.githubusercontent.com/dbabrams/ShallowDolomite_Group/master/Pumpage_Data_WillCounty.csv?token=AOLJKS4QB2KKXHBNRWG5UP26UCVUO'

#Create a copy of the data is not edited
#This will be needed for McKaleigh for 3d
raw_data = pd.read_csv(raw_csv)
print(raw_data)

print('size of original dataframe')
print(raw_data.shape)

      p_num  isws_facility_id  \
0    152530        19713228.0   
1    152562        19713228.0   
2    158221        19714241.0   
3    223202        19714241.0   
4    224255        19714515.0   
5    224256        19714515.0   
6    292193        19713940.0   
7    304771        19713228.0   
8    338236        19770000.0   
9    345572        19713330.0   
10   359321        19795400.0   
11   382164        19770000.0   
12   400038        19790450.0   
13   400177        19790550.0   
14   400178        19790650.0   
15   400180               NaN   
16   400181        19790450.0   
17   400182        19790900.0   
18   400183        19790250.0   
19   400184        19790250.0   
20   400185        19790250.0   
21   400186        19790500.0   
22   400187        19790500.0   
23   400188        19790800.0   
24   400189        19790800.0   
25   400190        19794150.0   
26   401404         4314862.0   
27   401405         4314862.0   
28   401500         4316928.0   
29   40162

In [10]:
#This copy of the data we will edit heavily 
df1 = pd.read_csv(raw_csv)

#drop rows if the depth, lamx, or lamy is unknown
df1.dropna(subset=['depth_total_last_known', 'lam_x', 'lam_y'], inplace = True)
    
#drop all of the rows that are deep wells (>400 ft)
df1.drop(df1[df1['depth_total_last_known'] > 400].index, inplace = True) 

print('size of updated dataframe')
print(df1.shape)


size of updated dataframe
(336, 46)


In [11]:
#df2 provides all of the old data but combined by facility; small fry facilities (<0.1mgd) were removed
#reset index to be based on well owner
df2=df1.set_index('owner')
#delete extra rows so that the dataframe will just be owner and the years of pumping
df2=df2.drop(['p_num', 'isws_facility_id', 'fac_well_num','depth_total_last_known', 'lam_x', 'lam_y'], axis=1)
#sum together all pumping for the same well owners so that the dataset is by pumping by facility
df2=df2.groupby(level=0).sum(min_count=1)

#calculate the maximum pumping for each facility
df2['max']=df2[['1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']].max(axis=1) 
#remove all facilities with nan in max and where maximum well pumping is less than 0.1mgd / "small fry" wells
#df3 drops all rows where the max value is < 0 and wells that don't meet minimum pumpage, then these calculation columns are removed in df4
df3=df2.dropna(subset=['max'],axis=0)
df3['small']=df3['max']-36524250
df4=df3[df3.small > 0]
df4=df4.drop(['small', 'max'], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [12]:
#loop to calculate the average of each row and then turn any columns with values greater than twice the average to nan
for index, row in df4.iterrows():
    ave = df4.mean(axis=1)
    df4['ave']=df4.index.map(ave)
    for columns in df4.columns:
        df4[columns][df4[columns] > 2*df4.ave]=np.nan
        print(df4[columns])

Streaming output truncated to the last 5000 lines.
AQUA ILLINOIS - OAKVIEW                              NaN
BEECHER                                              NaN
BOLINGBROOK                                          NaN
BONNIE BRAE - FOREST MANOR SAN DIST                  NaN
BROKEN ARROW GOLF COURSE                             NaN
CENTRAL SOD FARMS - WILL CTY                         NaN
CLEARVIEW SUBD                                       NaN
CREST HILL                                           NaN
CRETE                                         74950000.0
CRETE-MONEE HIGH SCHOOL                              NaN
CRYSTAL LAWNS ADDITION IMPROVEMENT ASSOC.     21704800.0
DEER CREEK GOLF COURSE                               NaN
DESOTO LLC                                           NaN
ELWOOD                                               NaN
EVERGREEN SOD FARM                                   NaN
FRANKFORT                                            NaN
HANSON MATERIAL SERVICE - YARD 61    

In [14]:
#remove the more recent years
subset = df4[['2013', '2014', '2015', '2016', '2017', '2018', '2019']]

#drop the orignal recent years from the dataframe
modify3 = df4.drop(['2013', '2014', '2015', '2016', '2017', '2018', '2019'], axis='columns')

#to bridge the more recent data, forward fill, back fill, and subbing in 0s
subset.fillna(method='ffill', axis='columns', inplace=True, limit=3, downcast=None)
subset.fillna(method='bfill', axis='columns', inplace=True, limit=3, downcast=None)
subset.fillna(value = 0, axis=1, inplace=True)

#replace updated recent years and forming a new dataframe
df5 = pd.concat([modify3, subset], axis = 1)

#fill in for up to two years of lapsed reporting data, the rest NaNs become 0
df5 = df5.fillna(method='ffill', axis='columns', limit=1)
df5 = df5.fillna(method='bfill', axis='columns', limit=1)
df5 = df5.fillna(0)
print(df5)

                                                  1981          1982  \
owner                                                                  
AQUA ILLINOIS - OAKVIEW                      7859520.0  9.028800e+06   
BEECHER                                     77559000.0  7.845280e+07   
BOLINGBROOK                                180009000.0  2.164300e+08   
BONNIE BRAE - FOREST MANOR SAN DIST         56047000.0  5.356400e+07   
BROKEN ARROW GOLF COURSE                           0.0  0.000000e+00   
CENTRAL SOD FARMS - WILL CTY                       0.0  0.000000e+00   
CLEARVIEW SUBD                               9125000.0  1.260000e+07   
CREST HILL                                 359102900.0  2.927235e+08   
CRETE                                      169664200.0  1.202365e+08   
CRETE-MONEE HIGH SCHOOL                      1095000.0  0.000000e+00   
CRYSTAL LAWNS ADDITION IMPROVEMENT ASSOC.   46360000.0  3.482784e+07   
DEER CREEK GOLF COURSE                             0.0  0.000000

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
